In [ ]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Auswerten eines Schach-Endspiels

Neben der Berechnung der Endspiel-Situationen gilt es zu überprüfen, ob es sich bei den Zügen der KI auch um optimale Züge handelt. Hierfür werden im Rahmen von diesem Notebook Testszenarien geschrieben, welche die Entscheidungen der KI bewerten. Zum Vergleich wurde die Stockfish-Engine herangezogen. Stockfish ist die momentan (stand 2022) beste Schach-Engine, die frei zur Verfügung steht. 

Insgesamt können vier unterschiedliche Szenarien in dem Notebook getestet werden. Die Struktur der Tests ist wie folgt aufgebaut:
* ``compare_fen_stockfish``: Dieser Test überprüft für eine mitgegebene FEN jeweils, wie viele Züge die Endspiel-Tabellen und Stockfish zum Gewinnen brauchen. 
* ``test_random_boards``: Dieser Test überprüft für eine gegebene Anzahl n zufälliger Situationen, ob Stockfish bessere Ergebnisse liefert.
* ``compare_sequence_stockfish``: Dieser Test überprüft für ein mitgegebenes $n$ alle Situationen, die sich in der Menge $S_n$ befinden, ob Stockfish oder die KI bessere Ergebnisse erzielt.
* ``compare_all_sequences``: Dieser Test überprüft für jegliche Situationen, die einer $S_n$ Menge zugeordnet wurden, ob Stockfish eine Lösung in weniger Zügen bestimmen kann.

Die Ergebnisse der letzten drei Tests werden im Ordner ``/Tests`` gespeichert.

## Konfiguration

Zu Beginn müssen für den Vergleich unterschiedliche Variablen definiert werden. Diese beinhalten den Namen der ``.chessTest`` Datei zu den berechneten $S_n$ Sequenzen und einen Pfad zur Stockfish-Installation. Weiterhin wird eine globale Variable ``VERBOSE`` festgelegt, die das Anzeigen von zusätzlichen Ausgaben ein oder ausschaltet.

In [ ]:
STOCKFISH_PATH = "./stockfish/stockfish.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

In [ ]:
S_N_FILE = "S_n_seq_queen"

S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens(S_N_FILE)

## Vergleich zwischen Stockfish und KI
Für den Vergleich einer Situation wird die Anzahl der Züge zwischen Stockfish und der selbst geschriebenen KI miteinander verglichen. Eine geringere Anzahl ist hierbei besser.

### Berechnung der Zuglisten
Um die Anzahl der benötigten Züge zu bestimmen, muss eine Liste angelegt werden, welche Züge durchgeführt werden. Dafür muss immer ein optimaler Zug bestimmt werden.
Diese Aufgabe wird von der Funktion ``find_next_move`` erfüllt. Diese berechnet für eine übergebene FEN (``fen``) den idealen Spielzug. Für die Berechnung wird der ``s_index`` mitgegeben, in dem die FEN gefunden werden kann. Diese wird für die Suche in den "gekürzten" übergebenen $S_n$ Mengen (``s_n_sequence_short``) verwendet. Die Suche findet mithilfe der bereits definierten Funktion ``find_situation_in_sequence`` statt.

Die Züge werden wie folgt bestimmt:
- `Stockfish`: Für sowohl schwarz als auch für weiß, werden die Züge mit der Funktion ``Stockfish().get_best_move()`` bestimmt.
- `KI`: Für Weiß werden die Züge anhand der $S_n$ Mengen bestimmt, die schwarzen Züge werden mit der Funktion ``Stockfish().get_best_move()`` gewählt.

Ein gegnerischer Spieler macht nicht immer einen im Sinne der KI optimalen Zug, und das Auswählen zufälliger Züge entspräche einem unerfahrenen Spieler. Um die Effizienz der KI gegen einen guten Spieler zu demonstrieren werden die Züge von Schwarz beim Bestimmen der Zuganzahl von Stockfish übernommen. Aus demselben Grund spielt Stockfish beim Bestimmen der Engine-Anzahl an Zügen in der folgenden Funktion `stockfish_movelist` gegen sich selbst. 

Die Funktion liefert als Rückgabewert den besten Zug ``move`` und der neue ``s_index``, welcher beschreibt, in welcher $S_n$ Menge die Spielsituation zu finden ist.

In [ ]:
def find_next_move(fen, s_index, s_n_sequence_short):
    STOCKFISH.set_fen_position(fen)
    curr_board = chess.Board(fen)
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            cur_fen = curr_board.fen()
            _tmp = find_situation_in_sequence(get_board_and_turn(cur_fen), [s_n_sequence_short[s_index - 1]])
            s_index_tmp = s_index - 1
            if _tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, move
            curr_board.pop()

        return -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        cur_fen = curr_board.fen()
        s_index = find_situation_in_sequence(get_board_and_turn(cur_fen), s_n_sequence_short[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, move

Bei der Berechnung der Züge für die KI wird die Funktion ``calculate_all_moves`` verwendet. Diese berechnet in einer Schleife alle Moves ausgehend von einer FEN, bis die KI gewonnen hat. Die Berechnung verwendet hierzu die FEN der Spielsituation (``fen``) und die Liste der gekürzten $S_n$ Mengen (``s_n_sequence_short``). Mithilfe von diesen Parametern wird eine Liste von Zügen bestimmt, in der die Partie zwischen Stockfish und der selbst geschriebenen KI beendet wurde.

In [ ]:
def calculate_all_moves(fen, s_n_sequence_short):
    moves = []

    s_index = find_situation_in_sequence(get_board_and_turn(fen), s_n_sequence_short)

    board = chess.Board(fen)
    while s_index > 0:
        cur_fen = board.fen()
        s_index, next_move = find_next_move(cur_fen, s_index, s_n_sequence_short)
        board.push(next_move)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

Nachdem eine Liste aller Züge für die KI bestimmt wurden, gilt dies gleichermaßen für die Berechnung von Stockfish umzusetzen. Hierfür erhält die Funktion ``stockfish_movelist`` eine ``fen`` mit der aktuellen Spielsituation. In der Funktion bestimmt Stockfish solange die bestmöglichen Züge, bis die Partie beendet worden ist. Diese Liste von Zügen wird am Ende der Funktion zurückgegeben.

In [ ]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves


### Hilfsfunktionen für den Vergleich

Der Vergleich zwischen der Stockfish Engine und der KI basiert auf der Anzahl der Züge, bis weiß das Spiel gewonnen hat. 
Hierfür werden drei Hilfsfunktionen definiert, die für den Vergleich benötigt werden.

Bei der ersten Funktion handelt es sich um ``compare_move_list``. Diese erhält eine Liste von 3-Tupeln(``move_count_list``), die als ersten Wert die Anzahl der Züge der KI beinhaltet und als zweiten Wert die Anzahl der Züge, die Stockfish zum Beenden der Partie benötigt hat. Der dritte Wert stellt die Differenz zwischen den beiden Anzahlen dar.

Als Ergebnis liefert die Funktion drei Werte zurück, die ein erstes Abbild für die Performanz der KI darstellen. Der erste Wert ist die Anzahl der Spielsituationen, bei denen Stockfish und die KI gleich viele Züge benötigt haben (``equal``). Der zweite Wert ist die Anzahl der Spiele, bei denen die KI weniger Züge benötigt hat (``ki_better``) und der letzte Werte die Anzahl der Spiele, in der Stockish weniger Züge benötigt hat (``stockfish_better``).

In [ ]:
def compare_move_lists(move_count_list):
    equal = 0
    ki_better = 0
    stockfish_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff == 0:
            equal += 1
        elif diff < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    return equal, ki_better, stockfish_better

Diese Werte können nun weiter genutzt werden, indem neben der Anzahl der Ergebnisse auch der Grad des Unterschieds bestimmt wird. Hierfür wurde die Funktion ``get_average_difference`` definiert. Diese nutzt ebenfalls die Liste von 3-Tupeln (``move_count_list``) als Input. Die Funktion berechnet aus dieser Liste die durchschnittliche Prozentzahl an Zügen, welche von Stockfish oder der KI weniger benötigt werden. Diese werden für sowohl die KI(``avg_ki_better``), als auch für Stockfish zurückgegeben(``avg_stock_better``).

In [ ]:
def get_average_difference(move_count_list):
    percentual_ki = []
    percentual_stock = []
    avg_ki_better = 0
    avg_stock_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff < 0:
            percentual_ki.append(round(1 - (ki_move / stock_move), 4))
        elif diff > 0:
            percentual_stock.append(round(1 - (stock_move / ki_move), 4))
    if len(percentual_ki) != 0:
        avg_ki_better = sum(percentual_ki) / len(percentual_ki)
    if len(percentual_stock) != 0:
        avg_stock_better = sum(percentual_stock) / len(percentual_stock)
    return avg_ki_better, avg_stock_better

Die Ergebnisse des Vergleichs sollen abschließend in einer Datei gespeichert werden. Dies hat den Grund, dass bei einer großen Anzahl von $S_n$ Mengen die Auswertung nicht in einem unübersichtlichen Konsolenfenster durchgeführt werden muss und man die Ergebnisse mehrfach betrachten kann. 
Hierfür wird die Funktion ``write_result_to_file`` definiert. Diese berechnet zunächst mit der ``move_count_list`` die Ergebnisse des Vergleichs. Hierzu werden die zuvor definierten Funktionen ``compare_move_lists`` und ``get_average_difference`` aufgerufen. Außerdem werden die prozentualen Ergebnisse, welche Engine/KI das Problem besser gelöst hat, innerhalb dieser Funktion berechnet. Die Ergebnisse werden letztendlich in die Datei mit dem Namen ``filename``.txt in den Ordner ``Tests`` geschrieben. Zur Übersicht wird ein ``sequence_index`` mitgegeben werden, der eine Unterscheidung der Tests ermöglicht.

In [ ]:
def write_result_to_file(filename, sequence_index, move_count_list):
    equal, ki_better, stockfish_better = compare_move_lists(move_count_list)
    avg_ki_better, avg_stock_better = get_average_difference(move_count_list)
    count = ki_better + stockfish_better + equal
    f = open("Tests/" + filename + ".txt", "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better / count) * 100, 2)) + "% besser.\n")
    f.write("Die KI war zu " + str(round((ki_better / count) * 100, 2)) + "% besser.\n")
    f.write("Stockfish und die KI haben zu " + str(
        round((equal / count) * 100, 2)) + "% die gleichen Ergebnisse erzielt.\n")
    f.write("Sofern die KI besser war, hat sie durchschnittlich " + str(
        round(avg_ki_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.write("Sofern Stockfish besser war, hat sie durchschnittlich " + str(
        round(avg_stock_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.close()

### Implementierung der Testszenarien
In diesem Abschnitt werden die zuvor definierten Testszenarien implementiert. Diese erhalten eine Liste der $S_n$ Mengen mit der verkürzten FEN Schreibweise (``s_n_sequence_short``) oder die Liste der $S_n$ Mengen mit der vollständigen FEN Schreibweise (``s_n_sequence_fen``). Diese werden für die Berechnungen der Züge für die KI benötigt und müssen deshalb in mindestens einer Form für die Funktion vorliegen. 

Das erste Testszenario sieht die Überprüfung eines Spielszenarios vor (``fen``). Dies geschieht in der Funktion ``compare_fen_stockfish``. Das Ergebnis wird am Ende der Funktion in der Konsole ausgegeben.

In [ ]:
def compare_fen_stockfish(fen, s_n_sequence_short):
    moves = calculate_all_moves(fen, s_n_sequence_short)
    stockfish_moves = stockfish_movelist(fen)

    if Moves is not None:
        print("AI needed " + str(len(moves)) + " moves to beat Stockfish as Black.")
    else:
        print("AI found no way to beat Black.")

    print("Stockfish needed " + str(len(stockfish_moves)) + " moves to win against itself.")

Das zweite Szenario sieht vor, eine beliebige Anzahl (``count``) zufälliger Boards zu vergleichen. Anhand der ``count``-Variable werden aus den $S_n$ Mengen zufällige Boards ausgewählt und der Vergleich zwischen der KI und Stockfish durchgeführt. Das Ergebnis wird mit ``write_result_to_file`` in einer Datei hinterlegt.

In [ ]:
import random


def test_random_boards(count, s_n_sequence_fen, s_n_sequence_short):
    count_s_n = len(s_n_sequence_fen)
    move_count_list = []
    for board_c in range(count):
        rand_sequence = random.randint(0, count_s_n - 1)
        rand_board = random.randint(0, len(s_n_sequence_fen[rand_sequence]) - 1)
        rand_fen = list(s_n_sequence_fen[rand_sequence])[rand_board]
        ki_moves = calculate_all_moves(rand_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(rand_fen)
        move_count = len(ki_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Analyzed " + str(board_c + 1) + "/" + str(count))

    filename = "Random_" + str(count) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                           "_") + ".txt"
    write_result_to_file(filename, "random", move_count_list)

In diesem Beispiel werden 100 zufällige Spielsituationen miteinander verglichen:

In [ ]:
test_random_boards(100, S_N_Sequence_fen, S_N_Sequence_fen_short)

Bei `compare_sequence_stockfish` wird für jedes Board einer $S_n$ Menge überprüft, ob Stockfish oder die zuvor definierten Mengen die Situation effizienter lösen. In Form des ``sequence_index`` wird das $n$ der $S_n$ Menge übergeben. Zusätzlich kann ein Name für eine Datei angegeben werden, sodass mehrere Testergebnisse in einer Datei festgehalten werden können. Das Beschreiben der Datei erfolgt mit der Funktion ``write_result_to_file``.

In [ ]:
def compare_sequence_stockfish(sequence_index, s_n_sequence_fen, s_n_sequence_short, g_filename=None):
    s_n = s_n_sequence_fen[sequence_index]
    move_count_list = []
    length = len(s_n)
    r = 0
    for board_fen in s_n:
        ai_moves = calculate_all_moves(board_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(board_fen)
        move_count = len(ai_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r + 1) + "/" + str(length))
        r += 1

    if g_filename is None:
        filename = "S_" + str(sequence_index) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                                   "_") + ".txt"
        write_result_to_file(filename, sequence_index, move_count_list)
    else:
        write_result_to_file(g_filename, sequence_index, move_count_list)

Als Beispiel wird hier der Vergleich zwischen der KI und Stockfish für die Menge $S_{20}$ vorgenommen:

In [ ]:
compare_sequence_stockfish(20, S_N_Sequence_fen, S_N_Sequence_fen_short)

Das letzte Testszenario sieht vor, alle zuvor bestimmten Boards mit dem Lösungsweg von Stockfish zu vergleichen. Hierfür geht die Funktion `compare_all_sequences` alle $S_n$ Mengen durch und vergleicht diese jeweils mit der Lösung von Stockfish. Alle Ergebnisse werden in eine Datei geschrieben.

In [ ]:
def compare_all_sequences(s_n_sequence_fen, s_n_sequence_short):
    filename = "All_S_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":", "_") + ".txt"
    for i in range(len(s_n_sequence_fen)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_fen, s_n_sequence_short, filename)

In [ ]:
compare_all_sequences(S_N_Sequence_fen, S_N_Sequence_fen_short)

Für die visuelle Darstellung der Züge wird die Funktion ``show_movelist`` definiert. Diese zeigt auf einem ``chess.Board`` Objekt eine mitgegebene Liste von Zügen zeitverzögert an. Für die Darstellung erhält die Funktion die ``fen``, die die Spielsituation zu Beginn der Partie darstellt. Weiter muss eine Liste von Spielzügen (``moves``), die dargestellt werden sollen, der Funktion mitgegeben werden.

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)
